In [70]:
# Importaciones
import json
import openai
import os
import spotipy
from dotenv import dotenv_values

In [74]:
# Variables de entorno
config = dotenv_values(".env")
openai.api_key = openai.api_key = config["OPENAI_API_KEY"]
SPOTIFY_CLIENT_ID = config["SPOTIFY_CLIENT_ID"]
SPOTIFY_CLIENT_SECRET = config["SPOTIFY_CLIENT_SECRET"]

0c37ecbb564040388cc9fccaec8019b8


In [66]:
# Obtener lista de canciones
def get_playlist(topic):

    prompt = f"""
        Necesito que me proporciones canciones infantiles cuya tematica sea: {topic} 
        Este contenido debe encontrarse en español, no debe repetirse
        Enfócate principalmente en que la canción contenga la temática que se quiere enseñar al niño.
        """


    context = """Eres un asistente útil para generar listas de reproducción.
                Deberías generar una lista de canciones y sus artistas según la indicación que se solicita.
                Debes ser claro, exacto y basarte en la información que contengas almacenada.
                Deberás devolver un array JSON, donde cada elemento sigue este formato:
                {"song": <song_title>, "artist": <artist_name>}
                """

    content = f"Generar una lista de 5 canciones basadas en este prompt: {prompt}"
    
    messages = [
        { "role": "system", "content": context},
        { "role": "user", "content": content, }
    ]

    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)

    playlist = json.loads(response.choices[0].message.content)
    return playlist

In [69]:
# Funcion para generar la imagen de la playlist
# TODO

In [67]:
# Crear lista de reproduccion
def add_songs_to_spotify(topic, playlist):

    sp = spotipy.Spotify( auth_manager=spotipy.SpotifyOAuth( client_id= SPOTIFY_CLIENT_ID, 
                                                            client_secret= SPOTIFY_CLIENT_SECRET, 
                                                            redirect_uri="http://localhost:9999", 
                                                            scope="playlist-modify-private"))

    current_user = sp.current_user()

    assert current_user is not None

    track_uris = []
    for item in playlist:
        artist, song = item["artist"], item["song"]

        advanced_query = f"artist:({artist}) track:({song})"
        basic_query = f"{song} {artist}"

        for query in [advanced_query, basic_query]:
            search_results = sp.search(
                q=query, limit=10, type="track"
            )

            if (
                not search_results["tracks"]["items"]
                or search_results["tracks"]["items"][0]["popularity"] < 20
            ):
                continue
            else:
                good_guess = search_results["tracks"]["items"][0]
                print(f"Encontrada: {good_guess['name']} [{good_guess['id']}]")
                track_uris.append(good_guess["id"])
                break

        else:
            print(
                f"Consultas {advanced_query} y {basic_query} no devuelve buenos resultados. Ignorando."
            )

    created_playlist = sp.user_playlist_create(
        current_user["id"],
        public=False,
        name=f"{topic}",
    )

    sp.user_playlist_add_tracks(current_user["id"], created_playlist["id"], track_uris)

    print("\n")
    print(f"Lista de reproducción creada: {created_playlist['name']}")
    print(created_playlist["external_urls"]["spotify"])

In [ ]:
# Main
topic = input("Seleccione el tema a tratar: ") #"El abecedario"
print(f"Creando playlist del tema elegido: {topic}...")
playlist = get_playlist(topic)

add_songs_to_spotify(topic, playlist)